In [63]:
!pip install -U selenium

In [6]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import NoSuchElementException

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException,ElementClickInterceptedException



In [7]:
def click_until_interactable(element_xpath):
    max_attempts = 10
    for attempt in range(max_attempts):
        try:
            wait = WebDriverWait(driver, 10)

            element = wait.until(EC.element_to_be_clickable((By.XPATH, element_xpath)))
            driver.execute_script("arguments[0].scrollIntoView();", element)
            element.click()
            return True
        except ElementClickInterceptedException:
            time.sleep(2)  # Wait for a while before retrying
    return False
        

In [ ]:
path = "C:\Chrome_Driver\chromedriver-win32\chromedriver.exe"
s = Service(path)
options = webdriver.ChromeOptions()
options.add_experimental_option("detach",True)
driver = webdriver.Chrome(service=s,options=options)
url="https://www.imdb.com/search/title/?title_type=feature,tv_series"
response=driver.get(url)
soup=BeautifulSoup(driver.page_source,'html.parser')
Scroll_count=50

for i in range(Scroll_count):
    
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")

    button_xpath ='//button[contains(@class,"ipc-see-more__button")]'
    wait = WebDriverWait(driver, 10)

    if not click_until_interactable(button_xpath):
        print("Failed to click the button after multiple attempts.")
    driver.execute_script("window.scrollTo(document.body.scrollHeight,document.body.scrollHeight/4)")

soup=BeautifulSoup(driver.page_source,'html.parser')


Failed to click the button after multiple attempts.
Failed to click the button after multiple attempts.


In [9]:
movie_dic={'name':[],
'img':[],
'release_year':[],
'duration':[],
'rating':[],
'reivew_count':[],
'cast':[],
'genre':[],
'director':[],
'desc':[]}

In [10]:
def get_mve_name(movie):
    try:
        mve_name=movie.find('h3',attrs={'class':'ipc-title__text'}).text
    except:
        mve_name=''
    return mve_name

def get_mve_img(movie):
    try:
        mve_img=movie.find('div',attrs={'class':'ipc-poster ipc-poster--base ipc-poster--dynamic-width ipc-sub-grid-item ipc-sub-grid-item--span-2'}).find('img',attrs={'class':'ipc-image'}).get('src')
    except:
        mve_img=''
    return mve_img
def get_mve_year(movie):
    try:
        mve_year=movie.find('span',attrs={'class':'sc-b189961a-8 kLaxqf dli-title-metadata-item'}).text
    except:
        mve_year=''
        
    return mve_year
def get_duration(movie):
    try:
        mve_duration=movie.find_all('span',attrs={'class':'sc-b189961a-8 kLaxqf dli-title-metadata-item'})[1].text
    except:
        mve_duration=''
    return mve_duration
def get_rating(movie):
    try:
        mve_rating=movie.find('span',attrs={'class':'ipc-rating-star ipc-rating-star--base ipc-rating-star--imdb ratingGroup--imdb-rating'}).text.replace('\xa0','')
    except:
        mve_rating=''
    return mve_rating
def get_vote(movie):
    try:
        mve_vote=movie.find('span',attrs={'ipc-rating-star--voteCount'}).text.replace('(','').replace(')','').replace('\xa0','')
    except:
        mve_vote=''
    return mve_vote
def get_desc(movie):
    try:
        mve_desc=movie.find('div',attrs={'class':'ipc-html-content-inner-div'}).text
    except:
        mve_desc=''
    return mve_desc
def get_director(movie_soup):
    try:
        mve_director=movie_soup.find('a',attrs={'class':'ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link'}).text
    except Exception as e:
        mve_director=''
    return mve_director
def get_genre(movie_soup):
    try:
        mve_genre=movie_soup.find_all('span',attrs={'class':'ipc-chip__text'})
        genres=[]
        for gen in mve_genre:
            genres.append(gen.text)
    except Exception as e:
        genres=''
    return genres
def get_stars(movie_soup):
    try:
        mve_star=movie_soup.find_all('a',attrs={'class':'ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link'})
        genres=[]
        
        for start in mve_star:
            genres.append(start.text)
    except Exception as e:
        genres=''
    return genres




In [11]:
movie_data=soup.find_all('div',attrs={'class':'ipc-metadata-list-summary-item__c'})
len(movie_data)

250

In [12]:
for movie in movie_data:
    movie_dic['name'].append(get_mve_name(movie))
    movie_dic['img'].append(get_mve_img(movie))
    movie_dic['release_year'].append(get_mve_year(movie))
    movie_dic['duration'].append(get_duration(movie))
    movie_dic['rating'].append(get_rating(movie))
    movie_dic['reivew_count'].append(get_vote(movie))
    movie_dic['desc'].append(get_desc(movie))
    links=movie.find_all('a',attrs={'class':'ipc-lockup-overlay ipc-focusable'})
    movie_links=[]
    for link in links:
        movie_links.append(link.get('href'))
    for movie_link in movie_links:
        movie_url='https://www.imdb.com'+movie_link
        movie_details=driver.get(movie_url)
        time.sleep(10)
        movie_soup=BeautifulSoup(driver.page_source, 'html.parser')
        movie_dic['director'].append(get_director(movie_soup))
        movie_dic['cast'].append(get_stars(movie_soup))
        movie_dic['genre'].append(get_genre(movie_soup))
        
        
    

In [13]:
df1 = pd.DataFrame(movie_dic)
df1


,name,img,release_year,duration,rating,reivew_count,cast,genre,director,desc
0,1. The Boys,https://m.media-amazon.com/images/M/MV5BYTY2Zj...,2019–,TV-MA,8.7(679K),679K,"[Eric Kripke, Karl Urban, Jack Quaid, Antony S...","[Action, Comedy, Crime, Back to top]",Eric Kripke,A group of vigilantes set out to take down cor...
1,2. House of the Dragon,https://m.media-amazon.com/images/M/MV5BM2QzMG...,2022–,TV-MA,8.4(397K),397K,"[Ryan J. Condal, George R.R. Martin, Matt Smit...","[Action, Adventure, Drama, Back to top]",Ryan J. Condal,An internal succession war within House Targar...
2,3. Inside Out 2,https://m.media-amazon.com/images/M/MV5BYTc1MD...,2024,1h 36m,7.9(50K),50K,"[Kelsey Mann, Meg LeFauve, Dave Holstein, Kels...","[Animation, Adventure, Comedy, Back to top]",Kelsey Mann,"Follows Riley, in her teenage years, encounter..."
3,4. Bridgerton,https://m.media-amazon.com/images/M/MV5BY2ZiOD...,2020–,TV-MA,7.4(186K),186K,"[Chris Van Dusen, Nicola Coughlan, Claudia Jes...","[Drama, Romance, Back to top]",Chris Van Dusen,The eight close-knit siblings of the Bridgerto...
4,5. The Acolyte,https://m.media-amazon.com/images/M/MV5BY2RlMm...,2024–,TV-14,3.4(75K),75K,"[Leslye Headland, Amandla Stenberg, Lee Jung-j...","[Action, Adventure, Drama, Back to top]",Leslye Headland,An investigation into a shocking crime spree p...
...,...,...,...,...,...,...,...,...,...,...
245,246. Tulsa King,https://m.media-amazon.com/images/M/MV5BYmI3N2...,2022–,TV-MA,8.0(73K),73K,"[Taylor Sheridan, Sylvester Stallone, Andrea S...","[Crime, Drama, Back to top]",Taylor Sheridan,"Following his release from prison, Mafia capo ..."
246,247. Mission: Impossible - Dead Reckoning Part...,https://m.media-amazon.com/images/M/MV5BYzFiZj...,2023,2h 43m,7.7(254K),254K,"[Christopher McQuarrie, Bruce Geller, Christop...","[Action, Adventure, Thriller, Back to top]",Christopher McQuarrie,Ethan Hunt and his IMF team must track down a ...
247,248. Mad Men,https://m.media-amazon.com/images/M/MV5BNTgxND...,2007–2015,TV-MA,8.7(262K),262K,"[Matthew Weiner, Jon Hamm, Elisabeth Moss, Vin...","[Drama, Back to top]",Matthew Weiner,A drama about one of New York's most prestigio...
248,249. Prison Break,https://m.media-amazon.com/images/M/MV5BMTg3NT...,2005–2017,TV-14,8.3(584K),584K,"[Paul T. Scheuring, Dominic Purcell, Wentworth...","[Action, Crime, Drama, Back to top]",Paul T. Scheuring,A structural engineer installs himself in a pr...


In [61]:
df1['release_year'].value_counts()

NameError: name 'df1' is not defined